In [ ]:
import numpy as np
import pandas as pd 
import anndata as ad
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps
from matplotlib.figure import figaspect
import os 
from mesa import ecospatial as eco
from scipy.stats import ranksums,ttest_ind
import math

## Figure output setting

In [ ]:
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Verdana', 'Geneva', 'Lucid', 'Arial', 'Helvetica', 'sans-serif']
plt.rcParams['svg.fonttype'] = 'none'

## Load metadata

In [ ]:
meta_data = pd.read_csv("/mnt/nfs/home/huayingqiu/hodgkinebvmibi/rebuttal/S2.1_MIBI-Table 1.csv")

In [ ]:
meta_data = meta_data[["FOV", "FOV size"]]

In [ ]:
meta_data

In [ ]:
meta_data.groupby("FOV size").size()

In [ ]:
fov_dict = {}

for _, row in meta_data.iterrows():
    size = row["FOV size"]
    fov_num = row["FOV"]

    if size not in fov_dict:
        fov_dict[size] = []

    fov_dict[size].append(fov_num)

In [ ]:
fov_dict

## Load annotated data

In [ ]:
data = pd.read_csv('/mnt/nfs/home/huayingqiu/Hodgkin_github/data/df_toipic_noID.csv')

data.columns


In [ ]:
data = data[['cellLabel', 'pointNum', 'centroidX', 'centroidY', 'Annotation', 'topic', 'ebv_status']]

data_topic0 = data[data['topic'] == 'Topic-0']

data_topic0

In [ ]:
data_topic0 = data_topic0.groupby('pointNum').filter(lambda x: len(x) >= 100)

data_topic0

## Scatter plot to check spatial location and type of cells

In [ ]:
color_dict = {
    'Tumor': '#E41A1C',
    'DC': '#377EB8',
    'B': '#F781BF',
    'CD4': '#4DAF4A',
    'CD8': '#984EA3',
    'M2': '#F7C173',
    'NK': '#FFFF33',
    'Endothelial': '#A65628',
    'M1': '#00FFFF',
    'Neutrophil': '#F6CEC1',
    'Treg': '#BEBADA',
    'Cytotoxic CD4': '#253D24',
    'Cytotoxic CD8': '#FF7F00'
}



# Get unique pointNum values
point_nums = sorted(data['pointNum'].unique())
n_points = len(point_nums)

# Calculate grid dimensions (trying to make it as square as possible)
n_cols = math.ceil(math.sqrt(n_points))
n_rows = math.ceil(n_points / n_cols)

# Create figure and subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(30, 30))
axes = axes.flatten() if n_points > 1 else [axes]  # Handle single subplot case

# Get unique annotation values for the legend
unique_annotations = data['Annotation'].unique()

# Plot each pointNum in its own subplot
for i, point_num in enumerate(point_nums):
    subset = data[data['pointNum'] == point_num]
    ebv_status = subset["ebv_status"].unique()[0]
    ax = axes[i]
    
    # Plot each annotation type with its own color
    for annotation in unique_annotations:
        annotation_subset = subset[subset['Annotation'] == annotation]
        if not annotation_subset.empty:
            ax.scatter(annotation_subset['centroidX'], 
                      -annotation_subset['centroidY'], 
                      color=color_dict.get(annotation, 'gray'),  # Default to gray if not in dict
                      label=annotation if i == 0 else "",  # Only add label in the first subplot
                      alpha=0.7,
                      s = 0.1)
    
    # Set titles and labels
    ax.set_title(f'pointNum {point_num}, {ebv_status}')
    ax.set_xlabel('centroidX')
    ax.set_ylabel('centroidY')
    ax.grid(True, alpha=0.3)

# Turn off any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

# Create a single legend for all subplots
# handles, labels = axes[0].get_legend_handles_labels()
# fig.legend(handles, labels, loc='right', bbox_to_anchor=(0.5, 0.98),
#           ncol=min(5, len(unique_annotations)), title="Cell Types")

# Add an overall title
plt.suptitle('Cell Locations by pointNum', fontsize=16, y=1.02)
plt.tight_layout()

# Show the plot
plt.show()

## EBV LUT

In [ ]:
pointNum_to_ebv = dict(zip(data['pointNum'], data['ebv_status']))
pointNum_to_ebv

## Color palette for cell types

In [ ]:
color_dict =  {
    'Tumor': '#E41A1C',
    'DC': '#377EB8',
    'B': '#F781BF',
    'CD4': '#4DAF4A',
    'CD8': '#984EA3',
    'M2': '#F7C173',
    'NK': '#FFFF33',
    'Endothelial': '#A65628',
    'M1': '#00FFFF',
    'Neutrophil': '#F6CEC1',
    'Treg': '#BEBADA',
    'Cytotoxic CD4': '#253D24',
    'Cytotoxic CD8': '#FF7F00'
}

fig, ax = plt.subplots()

for label, color in color_dict.items():
    ax.scatter([], [], c=color, label=label, marker='o',edgecolors='black', linewidths=0.5)

ax.legend(loc='center', ncol=1, fontsize='large')
ax.axis('off')
plt.show()

# Generate diversity maps 

In [ ]:
for key, item in fov_dict.items():
    print(key)
    img_scale = int(key.split("x")[0])
    scale = 8 * img_scale

    # Get unique pointNum values
    point_nums = item
    n_points = len(point_nums)

    # Calculate grid dimensions (trying to make it as square as possible)
    n_cols = math.ceil(math.sqrt(n_points))
    n_rows = math.ceil(n_points / n_cols)

    # Create figure and subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(2 * n_cols * img_scale, 2 * n_rows * img_scale))
    axes = axes.flatten() if n_points > 1 else [axes]  # Handle single subplot case

    # Plot each pointNum in its own subplot
    for i, point_num in enumerate(point_nums):

        subset = data[data['pointNum'] == point_num]
        ebv_status = subset["ebv_status"].unique()[0]

        ax = axes[i]

        patches_coordinates = eco.generate_patches(spatial_data=data,
                                            library_key="pointNum",
                                            library_id=point_num,
                                            scaling_factor=scale,
                                            spatial_key = ['centroidX','centroidY'])
        
        patch_indices, patches_comp = eco.calculate_diversity_index(spatial_data=data,
                                                                library_key='pointNum',
                                                                library_id=point_num,
                                                                spatial_key=['centroidX','centroidY'],
                                                                patches=patches_coordinates,
                                                                cluster_key='Annotation',
                                                                metric='Shannon Diversity', return_comp=True)
        
        heatmap_fig = eco.diversity_heatmap(spatial_data=data,
                                            library_key='pointNum',
                                            library_id=point_num,
                                            spatial_key=['centroidX','centroidY'],
                                            patches=patches_coordinates,
                                            heterogeneity_indices=patch_indices,
                                            tissue_only=False,
                                            plot=False,
                                            return_fig=True)
        
        ax.imshow(heatmap_fig, cmap = "bwr")
        
        # Add a colorbar for cellLabel values
        # plt.colorbar(scatter, ax=ax, label='cellLabel')
        
        # Set titles and labels
        ax.set_title(f'pointNum {point_num}, {ebv_status}')
        ax.set_xlabel('centroidX')
        ax.set_ylabel('centroidY')
        ax.grid(True, alpha=0.3)

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    # Add an overall title
    plt.suptitle(f'Diversity Index Map, scale = {scale}, image scale = {key}', fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)  # Make room for the suptitle

    output = f"../mesa_plots/{img_scale}_{scale}.svg"

    # Save plot
    plt.savefig(output, dpi=320, format = "svg", bbox_inches = "tight")

    # Show the plot
    plt.show()



In [ ]:
color_dict = {
    'Tumor': '#E41A1C',
    'DC': '#377EB8',
    'B': '#F781BF',
    'CD4': '#4DAF4A',
    'CD8': '#984EA3',
    'M2': '#F7C173',
    'NK': '#FFFF33',
    'Endothelial': '#A65628',
    'M1': '#00FFFF',
    'Neutrophil': '#F6CEC1',
    'Treg': '#BEBADA',
    'Cytotoxic CD4': '#253D24',
    'Cytotoxic CD8': '#FF7F00'
}


for key, item in fov_dict.items():

    img_scale = int(key.split("x")[0])

    # Get unique pointNum values
    point_nums = item
    n_points = len(point_nums)

    # Calculate grid dimensions (trying to make it as square as possible)
    n_cols = math.ceil(math.sqrt(n_points))
    n_rows = math.ceil(n_points / n_cols)

    # Create figure and subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(img_scale * n_cols * 2, img_scale * n_rows * 2))
    axes = axes.flatten() if n_points > 1 else [axes]  # Handle single subplot case

    # Get unique annotation values for the legend
    unique_annotations = data['Annotation'].unique()

    # Plot each pointNum in its own subplot
    for i, point_num in enumerate(point_nums):
        subset = data[data['pointNum'] == point_num]
        ebv_status = subset["ebv_status"].unique()[0]
        ax = axes[i]
        
        # Plot each annotation type with its own color
        for annotation in unique_annotations:
            annotation_subset = subset[subset['Annotation'] == annotation]
            if not annotation_subset.empty:
                ax.scatter(annotation_subset['centroidX'], 
                        -annotation_subset['centroidY'], 
                        color=color_dict.get(annotation, 'gray'),  # Default to gray if not in dict
                        label=annotation if i == 0 else "",  # Only add label in the first subplot
                        alpha=0.7,
                        s = 3/img_scale)
        
        # Set titles and labels
        ax.set_title(f'pointNum {point_num}, {ebv_status}')
        ax.set_xlabel('centroidX')
        ax.set_ylabel('centroidY')
        ax.grid(True, alpha=0.3)

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    # Create a single legend for all subplots
    # handles, labels = axes[0].get_legend_handles_labels()
    # fig.legend(handles, labels, loc='right', bbox_to_anchor=(0.5, 0.98),
    #           ncol=min(5, len(unique_annotations)), title="Cell Types")

    # Add an overall title
    plt.suptitle(f'Cell Locations by pointNum, {key}', fontsize=16, y=1.02)
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
gdi_dict = {}
for key, item in fov_dict.items():
    sub_data = data[data["pointNum"].isin(item)]
    img_scale = int(key.split("x")[0])
    scale = 8 * img_scale
    region_to_condition = dict(zip(sub_data['pointNum'], sub_data['ebv_status']))

    gdi_results = eco.calculate_GDI(spatial_data=sub_data,
                                    scale=scale,
                                    library_key='pointNum',
                                    library_id=item,
                                    spatial_key=['centroidX','centroidY'],
                                    cluster_key='Annotation',
                                    hotspot=True,
                                    restricted=False,
                                    metric='Shannon Diversity')

    gdi_results['pointNum'] = region_to_condition.keys()
    gdi_results['ebv_status'] = gdi_results.index.map(region_to_condition)

    gdi_dict[key] = gdi_results

    

In [ ]:
gdi_df = pd.concat([item for key, item in gdi_dict.items()], axis = 0)

In [ ]:
gdi_df

In [ ]:
plt.figure(figsize=(6, 4))
ax = sns.boxplot(data=gdi_df, x="ebv_status", y="GDI")
sns.stripplot(data=gdi_df, x="ebv_status", y="GDI", alpha = 0.7)
sns.despine(offset=10, trim=True)

In [ ]:
plt.figure(figsize=(6, 4))

# Create violin plot with custom colors
violin = sns.violinplot(
    data=gdi_df, 
    x="ebv_status", 
    y="GDI",
    palette={"Positive": "#D55E00", "Negative": "#009E73"},
    inner = None,
    scale="width"
)

# Add individual data points with stripplot
sns.stripplot(
    data=gdi_df, 
    x="ebv_status", 
    y="GDI", 
    alpha=0.7,
    jitter=True,
    color='black',
    size=4
)

# Set custom x and y axis labels
plt.xlabel("EBV Status", fontsize=12, fontweight='bold')  # Custom x-axis title
plt.ylabel("GDI", fontsize=12, fontweight='bold')  # Custom y-axis title

# Customize plot appearance
plt.title("GDI Distribution by EBV Status", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save plot as SVG with editable text
plt.savefig("../mesa_plots/gdi_violin_plot.svg", format="svg", dpi=300, bbox_inches="tight", transparent=True)

# Show the plot
plt.show()

In [ ]:
d1 = gdi_df.loc[gdi_df.ebv_status == 'Positive','GDI']
d2 = gdi_df.loc[gdi_df.ebv_status == 'Negative','GDI']


In [ ]:
stat, p = ranksums(d1, d2)
p

In [ ]:
t_stat, p_value = stats.ttest_ind(d1, d2, equal_var=False)
p_value

In [ ]:
dpi_dict = {}
for key, item in fov_dict.items():
    sub_data = data[data["pointNum"].isin(item)]
    img_scale = int(key.split("x")[0])
    scale = 8 * img_scale
    region_to_condition = dict(zip(sub_data['pointNum'], sub_data['ebv_status']))

    dpi_results = eco.calculate_DPI(spatial_data=sub_data,
                                    scale=scale,
                                    library_key='pointNum',
                                    library_id=item,
                                    spatial_key=['centroidX','centroidY'],
                                    cluster_key='Annotation',
                                    hotspot=True,
                                    restricted=False,
                                    metric='Shannon Diversity')

    dpi_results['pointNum'] = region_to_condition.keys()
    dpi_results['ebv_status'] = dpi_results.index.map(region_to_condition)

    dpi_dict[key] = dpi_results


In [ ]:
dpi_df = pd.concat([item for key, item in dpi_dict.items()], axis = 0)

In [ ]:
dpi_df

In [ ]:
# Create the boxplot
plt.figure(figsize=(6, 4))
ax = sns.boxplot(data=dpi_df, x="ebv_status", y="DPI")
sns.stripplot(data=dpi_df, x="ebv_status", y="DPI", alpha = 0.7)
sns.despine(offset=10, trim=True)

In [ ]:
plt.figure(figsize=(6, 4))

# Create violin plot with custom colors
violin = sns.violinplot(
    data=dpi_df, 
    x="ebv_status", 
    y="DPI",
    palette={"Positive": "#D55E00", "Negative": "#009E73"},
    inner = None,
    scale="width"
)

# Add individual data points with stripplot
sns.stripplot(
    data=dpi_df, 
    x="ebv_status", 
    y="DPI", 
    alpha=0.7,
    jitter=True,
    color='black',
    size=4
)

# Set custom x and y axis labels
plt.xlabel("EBV Status", fontsize=12, fontweight='bold')  # Custom x-axis title
plt.ylabel("DPI", fontsize=12, fontweight='bold')  # Custom y-axis title

# Customize plot appearance
plt.title("DPI Distribution by EBV Status", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save plot as SVG with editable text
plt.savefig("../mesa_plots/dpi_violin_plot.svg", format="svg", dpi=300, bbox_inches="tight", transparent=True)

# Show the plot
plt.show()

In [ ]:
d1 = dpi_df.loc[dpi_df.ebv_status == 'Positive','DPI']
d2 = dpi_df.loc[dpi_df.ebv_status == 'Negative','DPI']


In [ ]:
stat, p = ranksums(d1, d2)
p

In [ ]:
t_stat, p_value = stats.ttest_ind(d1, d2, equal_var=False)
p_value